In [4]:
import pandas as pd
import numpy as np
# Create a 10-row DataFrame
data = {
    'Column1': range(1, 11),
    'Column2': range(11, 21),
    'Column3': range(21, 31)
}

df = pd.DataFrame(data)
df.head()

split_dataframes = np.array_split(df,10)

In [7]:
for idx, df in enumerate(split_dataframes, start = 3):
  print(df.head(1))

   Column1  Column2  Column3
0        1       11       21
   Column1  Column2  Column3
1        2       12       22
   Column1  Column2  Column3
2        3       13       23
   Column1  Column2  Column3
3        4       14       24
   Column1  Column2  Column3
4        5       15       25
   Column1  Column2  Column3
5        6       16       26
   Column1  Column2  Column3
6        7       17       27
   Column1  Column2  Column3
7        8       18       28
   Column1  Column2  Column3
8        9       19       29
   Column1  Column2  Column3
9       10       20       30


In [1]:
split_dataframes = pd.Dataframe([0,1,2,3,4,5,6,7,8,9])
for i in enumerate(split_dataframes, start = 3):
  print(i)

(3, 0)
(4, 1)
(5, 2)
(6, 3)
(7, 4)
(8, 5)
(9, 6)
(10, 7)
(11, 8)
(12, 9)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flair
from flair.models import TextClassifier
from flair.data import Sentence

import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 84.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 48.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
#import spacy summaries
datapath = '/content/drive/MyDrive/W266 Notes/W266 Final Project/textrank_summary.csv'
textrank_reviews = pd.read_csv(datapath, header=0)

In [4]:
textrank_reviews.head()

,business_id,text_top10,spacy_summary,textrank_summary
0,-09Oc2D14vRnmirPh0vlXw,[953]\n7/2019\n\nThis Cracker Barrel is locate...,"The food finally makes it to the table, and by...","Upon arriving, it was packed. But, you probabl..."
1,-0FX23yAacC4bbLaGPvyxw,As soon as we walked in to Greystone it was ob...,This would be a terrific place for lunch if on...,"Again though, no biggie. However, we were too ..."
2,-0TffRSXXIlBYVbb5AwfTg,Came here for the lunch special menu a little ...,I'd really love to come back to try their dinn...,"My first time here, I came with a group of 5. ..."
3,-0Ym1Wg3bXd_TDz8JtvOQg,We came here on a nice sunny afternoon to grab...,I think the next time I might just go for the ...,It was very delicious. This is a Filipino-styl...
4,-0gRYq5UjMtZbELj0KHxzA,I was with a couple of family members visiting...,The best part in my opinion is all the coffee ...,Not free. None. Very well done.


In [5]:
textrank_reviews['cleaned_summary'] = textrank_reviews['textrank_summary'].str.lower()

In [6]:
def flair_sentiment(input_string):
  '''Uses English sentiment classifier from Flair NLP library to calculate sentiment % for a string'''
  classifier = TextClassifier.load('en-sentiment')
  sentence = Sentence(input_string)
  classifier.predict(sentence)

  sentiment = sentence.labels[0].value
  score = round(sentence.labels[0].score,3)
  return sentiment, score

In [7]:
split_dataframes = np.array_split(textrank_reviews,10)

#run flair on each split df and save into folder path
i = 1
for idx, df in enumerate(split_dataframes, start = 1):
  df[['textrank_sentiment', 'textrank_score']] = df['cleaned_summary'].apply(flair_sentiment).apply(pd.Series)
  filename = "df"+str(i)+"_textrank.csv"
  df.to_csv('/content/drive/MyDrive/W266 Notes/W266 Final Project/partial files_textrank/'+filename, index = False)
  i += 1
  print(filename)


2023-07-30 05:28:55,482 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpk_3pm1z9


100%|██████████| 253M/253M [00:12<00:00, 21.6MB/s]

2023-07-30 05:29:08,236 copying /tmp/tmpk_3pm1z9 to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2023-07-30 05:29:09,020 removing temp file /tmp/tmpk_3pm1z9


df1_textrank.csv
df2_textrank.csv
df3_textrank.csv
df4_textrank.csv
df5_textrank.csv


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d2f092e7-f6e7-45b4-a79f-a0ce56944f85)')' thrown while requesting HEAD https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 6aec3290-131c-4ac3-a10b-d9564cd91ffb)')' thrown while requesting HEAD https://huggingface.co/distilbert-base-uncased/resolve/main/preprocessor_config.json
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 153ebd27-16d9-4368-b0d9-3f635daf976f)')' thrown while requesting HEAD https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b7f22523-419f-4f41-a44b-711c021f04f3)')' 

df6_textrank.csv
df7_textrank.csv
df8_textrank.csv
df9_textrank.csv
df10_textrank.csv
